<a href="https://colab.research.google.com/github/jiheddachraoui/Covid19-ultrasound-Compressed-video-based-Frame-level-Scoring/blob/main/frame_level_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# scoring using Convolutional Neural Network (CNN)

## Import The Required Libraries

In [ ]:
# basics
import sys
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import cv2
import pickle 
from PIL import Image
import shutil
# Keras Libraries
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from tensorflow.keras.utils import load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from keras.utils import  load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import keras.backend
import os
import random
import time
import datetime
from tensorflow.python.framework import ops
import inspect
import gc

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## utility functions

In [ ]:
# print date and time for given type of representation
def date_time(x):
    if x==1:
        return 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==2:    
        return 'Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==3:  
        return 'Date now: %s' % datetime.datetime.now()
    if x==4:  
        return 'Date today: %s' % datetime.date.today() 

In [ ]:
# reset tensorflow graph tp free up memory and resource allocation 
def reset_graph(model=None):
    if model:
        try:
            del model
        except:
            return False
    
   
    ops.reset_default_graph()
    K.clear_session()
    
    gc.collect()
    
    return True


# reset callbacks 
def reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint = None
    reduce_lr = None
    early_stopping = None
    tensorboard = None

In [ ]:
def create_directory(directory_path, remove=False):
    if remove and os.path.exists(directory_path):
        try:
            shutil.rmtree(directory_path)
            os.mkdir(directory_path)
        except:
            print("Could not remove directory : ", directory_path)
            return False
    else:
        try:
            os.mkdir(directory_path)
        except:
            print("Could not create directory: ", directory_path)
            return False
        
    return True

# Removes directory, if directory exists 
def remove_directory(directory_path):
    if os.path.exists(directory_path):
        try:
            shutil.rmtree(directory_path)
        except:
            print("Could not remove directory : ", directory_path)
            return False
        
    return True

def clear_directory(directory_path):
    dirs_files = os.listdir(directory_path)
    
    for item in dirs_files:
#         item_path = os.path.join(directory_path, item)
        item_path = directory_path+ item
        
        try:
            if os.path.isfile(item_path):
                os.unlink(item_path)
            elif os.path.isdir(item_path): 
                shutil.rmtree(item_path)
        except Exception as e:
            print(e)
            
    return True


def remove_empty_folders(path, removeRoot=True):
    if not os.path.isdir(path):
        return
    
    # remove empty subfolders
    files = os.listdir(path)
    
    if len(files):
        for f in files:
            fullpath = os.path.join(path, f)
            
            if os.path.isdir(fullpath):
                remove_empty_folders(fullpath)

    # if folder empty, delete it
    files = os.listdir(path)
    
    if len(files) == 0 and removeRoot:
        print("Removing empty folder:", path)
        os.rmdir(path)
        
        
def dir_file_count(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

In [ ]:
def create_charts(cnn, cnn_model):
    ## Define train & validation loss
    train_loss = cnn_model.history['loss']
    val_loss = cnn_model.history['val_loss']
    
    ## Define train & validation AUC
    train_auc_name = list(cnn_model.history.keys())[3]
    val_auc_name = list(cnn_model.history.keys())[1]
    train_auc = cnn_model.history[train_auc_name]
    val_auc = cnn_model.history[val_auc_name]
    
    ## Define y_pred & y_true
    y_true = test_generator.classes
    Y_pred = cnn.predict_generator(test_generator, steps = len(test_generator))
    y_pred = (Y_pred > 0.5).T[0]
    y_pred_prob = Y_pred.T[0]
    
    ## PLOT ##
    fig = plt.figure(figsize=(13, 10))
    
    ## PLOT 1: TRAIN VS. VALIDATION LOSS 
    plt.subplot(2,2,1)
    plt.title("Training vs. Validation Loss")
    plt.plot(train_loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()

    ## PLOT 2: TRAIN VS. VALIDATION AUC
    plt.subplot(2,2,2)
    plt.title("Training vs. Validation AUC Score")
    plt.plot(train_auc, label='training auc')
    plt.plot(val_auc, label='validation auc')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()
    
    ## PLOT 3: CONFUSION MATRIX
    plt.subplot(2,2,3)
      # Set up the labels for in the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    names = ['True Negatives', 'False Positives', 'False Negatives', 'True Positives']
    counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(names, percentages)]
    labels = np.asarray(labels).reshape(2,2)
    ticklabels = ['Normal', 'Pneumonia']

      # Create confusion matrix as heatmap
    sns.set(font_scale = 1.4)
    ax = sns.heatmap(cm, annot=labels, fmt='', cmap='Oranges', xticklabels=ticklabels, yticklabels=ticklabels )
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.title("Confusion Matrix") #plt.title("Confusion Matrix\n", fontsize=10)
    plt.xlabel("Predicted", size=14)
    plt.ylabel("Actual", size=14) 
    #plt.savefig('cm.png', transparent=True) 
    
    ## PLOT 4: ROC CURVE
    plt.subplot(2,2,4)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    auc = roc_auc_score(y_true, y_pred_prob)
    plt.title('ROC Curve')
    plt.plot([0, 1], [0, 1], 'k--', label = "Random (AUC = 50%)")
    plt.plot(fpr, tpr, label='CNN (AUC = {:.2f}%)'.format(auc*100))
    plt.xlabel('False Positive Rate', size=14)
    plt.ylabel('True Positive Rate', size=14)
    plt.legend(loc='best')
    #plt.savefig('roc.png', bbox_inches='tight', pad_inches=1)
    
    ## END PLOTS
    plt.tight_layout();
    
    ## Summary Statistics
    TN, FP, FN, TP = cm.ravel() # cm[0,0], cm[0, 1], cm[1, 0], cm[1, 1]
    accuracy = (TP + TN) / np.sum(cm) # % positive out of all predicted positives
    precision = TP / (TP+FP) # % positive out of all predicted positives
    recall =  TP / (TP+FN) # % positive out of all supposed to be positives
    specificity = TN / (TN+FP) # % negative out of all supposed to be negatives
    f1 = 2*precision*recall / (precision + recall)
    stats_summary = '[Summary Statistics]\nAccuracy = {:.2%} | Precision = {:.2%} | Recall = {:.2%} | Specificity = {:.2%} | F1 Score = {:.2%}'.format(accuracy, precision, recall, specificity, f1)
    print(stats_summary)

## code


In [ ]:
batch_size = 64
input_directory = "/content/gdrive/MyDrive/Dataset/grayscale_data/"
output_directory = "/content/gdrive/MyDrive/Dataset/output/"

train_path = input_directory + r"train"
val_path = input_directory + r"valid"
test_path = input_directory + r"test"

figure_directory = "output/figures" 
file_name_pred_batch = figure_directory+r"/result"
file_name_pred_sample = figure_directory+r"/sample"

main_model_dir = output_directory + r"models/"
main_log_dir = output_directory + r"logs/"


clear_directory(main_log_dir)
remove_empty_folders(main_model_dir, False)


model_dir = main_model_dir + time.strftime('%Y-%m-%d %H-%M-%S') + "/"
log_dir = main_log_dir + time.strftime('%Y-%m-%d %H-%M-%S')

create_directory(model_dir, remove=True)
create_directory(log_dir, remove=True)

model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"


Removing empty folder: /content/gdrive/MyDrive/Dataset/output/models/2023-01-09 19-54-54


In [ ]:
reset_graph()
reset_callbacks()

### callbacks

In [ ]:
print("Settting Callbacks at ", date_time(1))

checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True)


tensorboard = TensorBoard(
    log_dir=log_dir,
    batch_size=batch_size,
    update_freq = 'batch')


reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=5,
    cooldown=2,
    min_lr=0.0000000001,
    verbose=1)


#-----------------------------------------------------------------------------------------------------------------#
callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
# callbacks = [checkpoint, tensorboard]
#-----------------------------------------------------------------------------------------------------------------#
print("Set Callbacks at ", date_time(1))

Settting Callbacks at  Timestamp: 2023-01-09 20:11:05
Set Callbacks at  Timestamp: 2023-01-09 20:11:05


### preprosessing

In [ ]:
# Set a seed value
seed_value= 42

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value

os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value

random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
#tf.set_random_seed(seed_value)
tf.random.set_seed(seed_value)
# 5. For layers that introduce randomness like dropout, make sure to set seed values 
#model.add(Dropout(0.25, seed=seed_value))

#6 Configure a new global `tensorflow` session

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
#hyper_mode = 'grayscale'
#imagesize=(688, 1760, 3)
#imagesize=(688, 880,6)

In [ ]:
'''mess=[]
def convert_image_to_grayscale(image_path, output_folder):
    try:
      output_filename = os.path.basename(image_path)

    # Compute the full path of the output file
      output_path = os.path.join(output_folder, output_filename)

    # Check if the output file already exists
      if not os.path.exists(output_path):
    # Open the image
        image = Image.open(image_path)
        image = image.convert('L')
        #image = np.expand_dims(image, axis=-1)
        # Save the grayscale image
        image.save(output_path, 'JPEG')
        print(output_path)
      
    except:
      mess.append(image_path)
      
    # Convert the image to grayscale
   

def convert_folder_to_grayscale(folder_path, output_folder):
    # Get a list of all the files in the folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    filenames = os.listdir(folder_path)

    # Iterate over the files
    for filename in filenames:
        # Compute the full path of the file
        file_path = os.path.join(folder_path, filename)

        # Check if the file is an image
        if os.path.splitext(file_path)[1].lower() in ['.jpg', '.png']:
            # Convert the image to grayscale and save it to the output folder
            convert_image_to_grayscale(file_path, output_folder)

# Convert all images in the "input" directory to grayscale and save them to the "output" directory
s=['score0','score1','score2','score3']
for i in s:
  try:
    convert_folder_to_grayscale(r'/content/gdrive/MyDrive/Dataset/frames_dataset/valid/{}'.format(i), r'/content/gdrive/MyDrive/Dataset/grayscale_data/valid/{}'.format(i))
    #convert_folder_to_grayscale(r'/content/gdrive/MyDrive/Dataset/frames_dataset/test/{}'.format(i), r'/content/gdrive/MyDrive/Dataset/grayscale_data/test/{}'.format(i))
    convert_folder_to_grayscale(r'/content/gdrive/MyDrive/Dataset/frames_dataset/train/{}'.format(i), r'/content/gdrive/MyDrive/Dataset/grayscale_data/train/{}'.format(i))
  except Exception: 

    continue
print(mess)'''

'mess=[]\ndef convert_image_to_grayscale(image_path, output_folder):\n    try:\n      output_filename = os.path.basename(image_path)\n\n    # Compute the full path of the output file\n      output_path = os.path.join(output_folder, output_filename)\n\n    # Check if the output file already exists\n      if not os.path.exists(output_path):\n    # Open the image\n        image = Image.open(image_path)\n        image = image.convert(\'L\')\n        #image = np.expand_dims(image, axis=-1)\n        # Save the grayscale image\n        image.save(output_path, \'JPEG\')\n        print(output_path)\n      \n    except:\n      mess.append(image_path)\n      \n    # Convert the image to grayscale\n   \n\ndef convert_folder_to_grayscale(folder_path, output_folder):\n    # Get a list of all the files in the folder\n    if not os.path.exists(output_folder):\n        os.makedirs(output_folder)\n    filenames = os.listdir(folder_path)\n\n    # Iterate over the files\n    for filename in filenames:\n  

In [ ]:
'''for s in ['train','valid']:
    
    
    # Select a random sample of size 100 from the source directory
    path = r"/content/gdrive/MyDrive/Dataset/grayscale_data/{}".format(s)

    # List the files in the folder
    for root, dirs, filenames in os.walk(path, topdown=False):
    for subfolder in dirs:
        subfolder_path = os.path.join(root, subfolder)
        filenames = os.listdir(subfolder_path)

        # Calculate the number of files to keep
        num_files = int(len(filenames) * 0.4)

        # Randomly select a subset of the files to keep
        filenames_new = random.sample(filenames, num_files)

        # Delete the rest of the files
        for filename in filenames:
            p = os.path.join(subfolder_path, filename)
            if filename not in filenames_new:
                os.remove(p)
                print(p)'''

'for s in [\'train\',\'valid\']:\n    \n    \n    # Select a random sample of size 100 from the source directory\n    path = r"/content/gdrive/MyDrive/Dataset/grayscale_data/{}".format(s)\n\n    # List the files in the folder\n    for root, dirs, filenames in os.walk(path, topdown=False):\n    for subfolder in dirs:\n        subfolder_path = os.path.join(root, subfolder)\n        filenames = os.listdir(subfolder_path)\n\n        # Calculate the number of files to keep\n        num_files = int(len(filenames) * 0.4)\n\n        # Randomly select a subset of the files to keep\n        filenames_new = random.sample(filenames, num_files)\n\n        # Delete the rest of the files\n        for filename in filenames:\n            p = os.path.join(subfolder_path, filename)\n            if filename not in filenames_new:\n                os.remove(p)\n                print(p)'

In [ ]:


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.02,
                                   zoom_range = 0.02, 
                                 
                                   #preprocessing_function=preprocess_image ,
                                   horizontal_flip = False)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0,
                                                              
                                                               #preprocessing_function=preprocess_image 
                                                              ) 
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0,
                                                                
                                                               #preprocessing_function=preprocess_image
                                                               ) 


train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    
                                                    target_size = (240, 240), # image height , image width
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    #color_mode='grayscale',
                                                    color_mode='rgb',
                                                    shuffle=True,
                                                    seed=42)
val_generator = val_datagen.flow_from_directory(directory = val_path, 
                                                 
                                                    target_size = (240, 240), # image height , image width
                                                    class_mode="categorical",
                                                    #color_mode='grayscale',
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=42)
test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 
                                                    target_size = (240, 240), # image height , image width
                                                    class_mode="categorical",
                                                    #color_mode='grayscale',
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=42)

test_generator.reset()

Found 3502 images belonging to 4 classes.
Found 1457 images belonging to 4 classes.
Found 1073 images belonging to 4 classes.


In [ ]:
x,y = next(train_generator)
x.shape

(64, 240, 240, 3)

In [ ]:
'''a = train_generator.class_indices
class_names = list(a.keys())  # storing class/breed names in a list
 
 
def plot_images(img, labels):
    plt.figure(figsize=[15, 10])
    for i in range(batch_size):
        plt.subplot(5, 5, i+1)
        plt.imshow(img[i])
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis('off')
 
plot_images(x,y)'''

"a = train_generator.class_indices\nclass_names = list(a.keys())  # storing class/breed names in a list\n \n \ndef plot_images(img, labels):\n    plt.figure(figsize=[15, 10])\n    for i in range(batch_size):\n        plt.subplot(5, 5, i+1)\n        plt.imshow(img[i])\n        plt.title(class_names[np.argmax(labels[i])])\n        plt.axis('off')\n \nplot_images(x,y)"

## Model

In [ ]:
# load the InceptionResNetV2 architecture with imagenet weights as base
base_model = tf.keras.applications.resnet.ResNet50(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(240, 240,3),
                     
                    
                     )
 
base_model.trainable=False

# For freezing the layer we make use of layer.trainable = False
# means that its internal state will not change during training.
# model's trainable weights will not be updated during fit(),
# and also its state updates will not run.
 
cnn = tf.keras.Sequential([
        base_model 
         ])
"""  ,tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1000, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(120, activation='relu'),
        tf.keras.layers.Dense(60, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Dense(4, activation='softmax')"""
   

cnn.add(tf.keras.layers.Conv2D(1300, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
cnn.add(tf.keras.layers.Conv2D(600, (3, 3), padding="same", activation='relu'))




cnn.add(tf.keras.layers.Conv2D(600, (3, 3), activation='relu', padding="same"))
cnn.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'))

cnn.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

cnn.add(tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding="same"))
cnn.add(tf.keras.layers.Conv2D(64, (5,5), padding="same", activation='relu'))
#cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(5, 5)))
   
cnn.add(tf.keras.layers.Conv2D(32, (5, 5), activation='relu', padding="same"))
cnn.add(tf.keras.layers.Conv2D(32, (5, 5), padding="same", activation='relu'))


#cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation='relu'))
#cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

cnn.add(tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding="same", activation='relu'))


cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization(renorm=True))



cnn.add(Dense(16, activation='relu'))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(activation='relu', units=256))
cnn.add(tf.keras.layers.Dense(activation='relu', units=128))
cnn.add(tf.keras.layers.Dropout(0.4, seed=seed_value))
cnn.add(tf.keras.layers.Dense(activation='relu', units=64))
cnn.add(tf.keras.layers.Dense(activation='relu', units=16))
cnn.add(tf.keras.layers.Dense(activation='softmax', units=4))


'''for layer in cnn.layers:
  layer.trainable=True
'''






cnn.compile(optimizer=keras.optimizers.Adam(learning_rate=.01),loss='categorical_crossentropy',metrics=[AUC()])
# categorical cross entropy is taken since its used as a loss function for
# multi-class classification problems where there are two or more output labels.
# using Adam optimizer for better performance
# other optimizers such as sgd can also be used depending upon the model

early = tf.keras.callbacks.EarlyStopping( patience=10,
                                          min_delta=0.001,
                                          restore_best_weights=True)
checkpoint=checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/gdrive/MyDrive/Dataset/output/checkpoints/my_modelfra2_{epoch}.h5",
   save_best_only=True,
    verbose=1

)
reduce1=tf.keras.callbacks.ReduceLROnPlateau(factor=0.1,  # reduce the learning rate by a factor of 0.1
    patience=10,  # after 10 epochs with no improvement
    verbose=1)
callbacks=[early,checkpoint,reduce1]

# early stopping call back

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 1300)        23962900  
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 600)         7020600   
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 600)         3240600   
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         691328    
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 8, 128)         147584    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 128)        0

In [ ]:
"""from sklearn.utils import class_weight

def get_weight(t):
    class_weight_current =  class_weight.compute_class_weight('balanced', classes=np.unique(t), y=t)
    return class_weight_current

    
train_classes = train_generator.classes

class_weight =  get_weight(train_classes )
class_weight = dict(zip(np.unique(train_classes), class_weight))
class_weight"""


"from sklearn.utils import class_weight\n\ndef get_weight(t):\n    class_weight_current =  class_weight.compute_class_weight('balanced', classes=np.unique(t), y=t)\n    return class_weight_current\n\n    \ntrain_classes = train_generator.classes\n\nclass_weight =  get_weight(train_classes )\nclass_weight = dict(zip(np.unique(train_classes), class_weight))\nclass_weight"

In [ ]:
#cnn = tf.keras.models.load_model('/content/gdrive/MyDrive/Dataset/output/checkpoints/my_modelfra_1.h5')

In [ ]:

epochs=100

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = val_generator.n//val_generator.batch_size

print("start Model Trainning", date_time(1))
# fit model
history = cnn.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs,
                    callbacks=callbacks,
                    verbose=1
                    )


print("Completed Model Trainning", date_time(1))

#create_charts(cnn, history)

start Model Trainning Timestamp: 2023-01-09 20:11:43
Epoch 1/100
25/54 [============>.................] - ETA: 17:29 - loss: 9128.6768 - auc: 0.5236

KeyboardInterrupt: ignored

In [ ]:
cnn.save("/content/gdrive/MyDrive/Dataset/output/models/Model_frame.h5")

## Visualization

In [ ]:
def create_charts(cnn, cnn_model):
    ## Define train & validation loss
    train_loss = cnn_model.history['loss']
    val_loss = cnn_model.history['val_loss']
    
    ## Define train & validation AUC
    train_auc_name = list(cnn_model.history.keys())[3]
    val_auc_name = list(cnn_model.history.keys())[1]
    train_auc = cnn_model.history[train_auc_name]
    val_auc = cnn_model.history[val_auc_name]
    
    ## Define y_pred & y_true
    y_true = test_generator.classes
    Y_pred = cnn.predict(test_generator, steps = len(test_generator))
    y_pred = (Y_pred > 0.5).T[0]
    y_pred_prob = Y_pred.T[0]
    
    ## PLOT ##
    fig = plt.figure(figsize=(13, 10))
    
    ## PLOT 1: TRAIN VS. VALIDATION LOSS 
    plt.subplot(2,2,1)
    plt.title("Training vs. Validation Loss")
    plt.plot(train_loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()

    ## PLOT 2: TRAIN VS. VALIDATION AUC
    plt.subplot(2,2,2)
    plt.title("Training vs. Validation AUC Score")
    plt.plot(train_auc, label='training auc')
    plt.plot(val_auc, label='validation auc')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()
    
    ## PLOT 3: CONFUSION MATRIX
    plt.subplot(2,2,3)
      # Set up the labels for in the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    names = ['True Negatives', 'False Positives', 'False Negatives', 'True Positives']
    counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(names, percentages)]
    labels = np.asarray(labels).reshape(2,2)
    ticklabels = ['Normal', 'Pneumonia']

      # Create confusion matrix as heatmap
    sns.set(font_scale = 1.4)
    ax = sns.heatmap(cm, annot=labels, fmt='', cmap='Oranges', xticklabels=ticklabels, yticklabels=ticklabels )
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.title("Confusion Matrix") #plt.title("Confusion Matrix\n", fontsize=10)
    plt.xlabel("Predicted", size=14)
    plt.ylabel("Actual", size=14) 
    #plt.savefig('cm.png', transparent=True) 
    
    ## PLOT 4: ROC CURVE
    plt.subplot(2,2,4)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    auc = roc_auc_score(y_true, y_pred_prob)
    plt.title('ROC Curve')
    plt.plot([0, 1], [0, 1], 'k--', label = "Random (AUC = 50%)")
    plt.plot(fpr, tpr, label='CNN (AUC = {:.2f}%)'.format(auc*100))
    plt.xlabel('False Positive Rate', size=14)
    plt.ylabel('True Positive Rate', size=14)
    plt.legend(loc='best')
    #plt.savefig('roc.png', bbox_inches='tight', pad_inches=1)
    
    ## END PLOTS
    plt.tight_layout();
    
    ## Summary Statistics
    TN, FP, FN, TP = cm.ravel() # cm[0,0], cm[0, 1], cm[1, 0], cm[1, 1]
    accuracy = (TP + TN) / np.sum(cm) # % positive out of all predicted positives
    precision = TP / (TP+FP) # % positive out of all predicted positives
    recall =  TP / (TP+FN) # % positive out of all supposed to be positives
    specificity = TN / (TN+FP) # % negative out of all supposed to be negatives
    f1 = 2*precision*recall / (precision + recall)
    stats_summary = '[Summary Statistics]\nAccuracy = {:.2%} | Precision = {:.2%} | Recall = {:.2%} | Specificity = {:.2%} | F1 Score = {:.2%}'.format(accuracy, precision, recall, specificity, f1)
    print(stats_summary)

In [ ]:
create_charts(cnn,history)

In [ ]:
# store results
acc = history.history['auc']
val_acc = history.history['val_auc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
 
# plot results
# accuracy
plt.figure(figsize=(10, 16))
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title(f'\nTraining and Validation Accuracy. \nTrain Accuracy:{str(acc[-1])}\nValidation Accuracy: {str(val_acc[-1])}')

In [ ]:
# loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title(f'Training and Validation Loss. \nTrain Loss:{str(loss[-1])}\nValidation Loss: {str(val_loss[-1])}')
plt.xlabel('epoch')
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
accuracy_score = cnn.evaluate(val_generator)
print(accuracy_score)
print("Accuracy: {:.4f}%".format(accuracy_score[1] * 100))
 
print("Loss: ",accuracy_score[0])

In [ ]:
test_img_path = "/content/gdrive/MyDrive/Dataset/data2/test/score0/convex_1048_1126_8.jpg"
 
img = cv2.imread(test_img_path)
resized_img = cv2.resize(img, (331, 331)).reshape(-1, 331, 331, 3)/255
 
plt.figure(figsize=(6,6))
plt.title("TEST IMAGE")
plt.imshow(resized_img[0])

In [ ]:
'''predictions = []
images=[]
for root, dirs, files in os.walk(test_path, topdown=False):
   for image in files:
      image=os.path.join(root, image)
      images.append(image)
      img = tf.keras.preprocessing.image.load_img(image)
      img = tf.keras.preprocessing.image.img_to_array(img)
      img = tf.keras.preprocessing.image.smart_resize(img, (688, 1760))
      img = tf.reshape(img, (-1, 688, 1760, 3))
      prediction = model.predict(img/255)
      predictions.append(np.argmax(prediction))
sample=pd.DataFrame(images)  

my_submission = pd.DataFrame({'image_id': images, 'label':  predictions})
if os.path.exists('/content/gdrive/MyDrive/Dataset/output/submission.csv'):
  os.remove('/content/gdrive/MyDrive/Dataset/output/submission.csv')
my_submission.to_csv('/content/gdrive/MyDrive/Dataset/output/submission.csv', index=False)
 
# Submission file ouput
print("Submission File: \n---------------\n")
print(my_submission.head()) # Displaying first five predicted output'''